In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import itertools as it

In [2]:
def separation(data):
    '''Function for separation of certain data in the dataset.
       Function takes in a string and returns a
       list of elements of the string'''
    if '/' in data:
        data = data.split('/')
    return data

def coma_separation(data):
    '''Function for separation of certain data in the dataset.
       Function takes in a string and returns a
       list of elements of the string'''
    data = str(data)
    if ',' in data:
        data = data.split(', ')
    return data

In [3]:
data = pd.read_csv('Sexuality_in_video_games_unedited.csv')

In [4]:
data

,Timestamp,Your Gender,Your Sexual Orientation,Your age,What country are You from?,How often do You play computer games,And for how long do Your gaming sessions usually last,Your favourite video game genre,Your favourite game(May write several),What other interests do You hold(You can choose multiple and write any others in Other),Your overall view on seeing sexual themes in the game(What is you reaction on seeing sexual themes in a game),Your view on sexualisation of the character design(Your reaction on seeing (overly)sexual character design),Any specific things that you usually LIKE in character designs,Any specific things that you usually DISLIKE in character designs,Your view on sexual sub-context in video games- themes around sex and similar,"If you dislike the (over)use of sexual sub-content, would you be okay if it's explained logically, tied to the story(And-Or if it exists for more reasons than just for marketing and-or similar)",Are you okay with LGBT+ characters in video games,Anything to add to the theme? Your personal thoughts.
0,04/01/2021 18:22:52,Prefer not to say,confused,27-35,Latvia,Every day,5-8 hours,Multiplayer RPGs,"WoW, Lol, Tekken","Books(Non asian), Tabletop games(just the basi...",Sometimes positive,4,NaN,Oversized breasts,3,Yes,"Yes, but only if explained logically and-or ex...",NaN
1,04/01/2021 18:43:27,Male,Heterosexual,21-26,Russia,Several times a week,0-4 hours,Immersive Sims,"Deus Ex: Human Revolution, Prey (2017), Mass E...","Books(Non asian), Films and TV series",Sometimes positive,4,When it makes sense in a given setting,When there is no sense or the explanation is w...,4,Yes,"Yes, but only if explained logically and-or ex...",There are moments when erotic scenes are just ...
2,04/01/2021 18:43:43,Male,Heterosexual,21-26,Russia,Several times a week,0-4 hours,Singleplayer RPGs,Red Dead Redemption 2,"Books(Non asian), Films and TV series, Manga",Mostly positive,7,NaN,NaN,5,Yes,"Yes, but only if explained logically and-or ex...",NaN
3,04/01/2021 18:52:43,Male,Heterosexual,27-35,Russian Federation,Every day,5-8 hours,Don't have favorite,"Ace Combat 7, SpecOps The Line, Deus Ex Human ...","Books(Non asian), Tabletop RPGs(D&D and simila...",Neutral,4,marks of their mind characteristics,"Extreme sexualisation, too bright colors, hype...",4,Yes,Maybe/Neutral,NaN
4,04/01/2021 19:04:46,Male,Heterosexual,21-26,Latvia,Ranges from every day to several times a week.,5-8 hours,Action,Devil May Cry; Dark Souls; Tekken; Escape From...,"Tabletop games(just the basic ones), Films and...",Sometimes positive,4,When the design of the character serves to tel...,Dislike when the creative process of designing...,4,Yes,If that type of content feels forced on then n...,As the LGBT+ community has gained more tractio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,05/01/2021 11:37:11,Male,Heterosexual,21-26,US,Several times a week,0-4 hours,Multiplayer RPGs,"Halo, need for speed: underground, overwatch","Books(Non asian), Films and TV series, Anime",Sometimes negative,3,NaN,NaN,4,Yes,"Yes, in any way they appear there",NaN
243,05/01/2021 11:37:39,Male,Heterosexual,More than 35,Germany,Several times a week,0-4 hours,Action,"Celeste, Doom, Super Mario Galaxy, Firewatch, ...","Books(Non asian), Tabletop games(just the basi...",Neutral,2,I like pretty faces and generally attractive (...,"exaggerated boobs, unrealisticly skimpy outfit...",5,"Maybe, but if it's nonsense like Quiet's outfi...","Yes, in any way they appear there","I like playing games with attractive people, b..."
244,05/01/2021 11:43:50,Female,Heterosexual,27-35,USA,Several times a week,0-4 hours,Singleplayer RPGs,Skyrim,"Books(Non asian), Films and TV series",Neutral,2,NaN,"Sexualized female characters, non sexualized m...",4,It is too overdone to be ok with context,"Yes, in any way they appear there",NaN
245,05/01/2021 11:45:41,Male,Heterosexual,Younger than 16,NaN,Several times a week,0-4 hours,Simulators,"Democracy 4, Ci

In [5]:
data.columns = ['date', 'Gender', 'Sexual_orientation', 
                'Age_group','Country','Regularity_of_gaming',
                'Session_length','Genre', 'Favourite_game','Other_interests',
               'View_on_sexuality','View_on_sexualisation','Design_preferences',
               'Design_issues','View_on_sexual_sub_context','bruh','LGBT_representation', 'extras']
data

,date,Gender,Sexual_orientation,Age_group,Country,Regularity_of_gaming,Session_length,Genre,Favourite_game,Other_interests,View_on_sexuality,View_on_sexualisation,Design_preferences,Design_issues,View_on_sexual_sub_context,bruh,LGBT_representation,extras
0,04/01/2021 18:22:52,Prefer not to say,confused,27-35,Latvia,Every day,5-8 hours,Multiplayer RPGs,"WoW, Lol, Tekken","Books(Non asian), Tabletop games(just the basi...",Sometimes positive,4,NaN,Oversized breasts,3,Yes,"Yes, but only if explained logically and-or ex...",NaN
1,04/01/2021 18:43:27,Male,Heterosexual,21-26,Russia,Several times a week,0-4 hours,Immersive Sims,"Deus Ex: Human Revolution, Prey (2017), Mass E...","Books(Non asian), Films and TV series",Sometimes positive,4,When it makes sense in a given setting,When there is no sense or the explanation is w...,4,Yes,"Yes, but only if explained logically and-or ex...",There are moments when erotic scenes are just ...
2,04/01/2021 18:43:43,Male,Heterosexual,21-26,Russia,Several times a week,0-4 hours,Singleplayer RPGs,Red Dead Redemption 2,"Books(Non asian), Films and TV series, Manga",Mostly positive,7,NaN,NaN,5,Yes,"Yes, but only if explained logically and-or ex...",NaN
3,04/01/2021 18:52:43,Male,Heterosexual,27-35,Russian Federation,Every day,5-8 hours,Don't have favorite,"Ace Combat 7, SpecOps The Line, Deus Ex Human ...","Books(Non asian), Tabletop RPGs(D&D and simila...",Neutral,4,marks of their mind characteristics,"Extreme sexualisation, too bright colors, hype...",4,Yes,Maybe/Neutral,NaN
4,04/01/2021 19:04:46,Male,Heterosexual,21-26,Latvia,Ranges from every day to several times a week.,5-8 hours,Action,Devil May Cry; Dark Souls; Tekken; Escape From...,"Tabletop games(just the basic ones), Films and...",Sometimes positive,4,When the design of the character serves to tel...,Dislike when the creative process of designing...,4,Yes,If that type of content feels forced on then n...,As the LGBT+ community has gained more tractio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,05/01/2021 11:37:11,Male,Heterosexual,21-26,US,Several times a week,0-4 hours,Multiplayer RPGs,"Halo, need for speed: underground, overwatch","Books(Non asian), Films and TV series, Anime",Sometimes negative,3,NaN,NaN,4,Yes,"Yes, in any way they appear there",NaN
243,05/01/2021 11:37:39,Male,Heterosexual,More than 35,Germany,Several times a week,0-4 hours,Action,"Celeste, Doom, Super Mario Galaxy, Firewatch, ...","Books(Non asian), Tabletop games(just the basi...",Neutral,2,I like pretty faces and generally attractive (...,"exaggerated boobs, unrealisticly skimpy outfit...",5,"Maybe, but if it's nonsense like Quiet's outfi...","Yes, in any way they appear there","I like playing games with attractive people, b..."
244,05/01/2021 11:43:50,Female,Heterosexual,27-35,USA,Several times a week,0-4 hours,Singleplayer RPGs,Skyrim,"Books(Non asian), Films and TV series",Neutral,2,NaN,"Sexualized female characters, non sexualized m...",4,It is too overdone to be ok with context,"Yes, in any way they appear there",NaN
245,05/01/2021 11:45:41,Male,Heterosexual,Younger than 16,NaN,Several times a week,0-4 hours,Simulators,"Democracy 4, Cities: Skylines","Books(Non asian), Tabletop games(just the basi...",Neutral,3,NaN,NaN,5,Yes,"Yes, in any way they appear there",NaN


In [6]:
def US_sort(Country):
    Country = str(Country)
    if Country.lower() == 'usa' or Country.lower() == 'united states ' or Country.lower() == 'murica' or Country.lower() == 'america' or Country.lower() == 'usa ' or  Country.lower() == 'us' or Country.lower() == 'united states' or Country.lower() == 'united states of america' or  Country.lower() == 'u.s.' :
        Country = 'USA'
        return Country
    else:
        return Country

def UK_sort(Country):
    Country = str(Country)
    if Country.lower() == 'uk' or Country.lower() == 'uk ' or  Country.lower() == 'united kingdom' or Country.lower() == 'england' or Country.lower() == 'scotland' or Country.lower() == 'united kingdom ' or Country.lower() == 'england ':
        Country = 'UK'
        return Country
    else:
        return Country
    
def Dutch_sort(Country):
    Country = str(Country)
    if Country.lower() == 'netherlands' or Country.lower() == 'the netherlands' or Country.lower() == 'netherlands ' or Country.lower() == 'the netherlands ':
        Country = 'The Netherlands'
        return Country
    else:
        return Country
    
    
def Canadian_sort(Country):
    Country = str(Country)
    if Country.lower() == 'canada' or Country.lower() == 'canada ':
        Country = 'Canada'
        return Country
    else:
        return Country

def French_sort(Country):
    Country = str(Country)
    if Country.lower() == 'france' or Country.lower() == 'france ':
        Country = 'France'
        return Country
    else:
        return Country
    
def Russian_sort(Country):
    Country = str(Country)
    if Country.lower() == 'russia' or Country.lower() == 'russian federation':
        Country = 'Russia'
        return Country
    else:
        return Country
    
def Oceanic_sort(Country):
    Country = str(Country)
    if Country.lower() == 'australia' or Country.lower() == 'new zealand' or Country.lower() == 'new zeland':
        Country = 'Australia and New Zealand'
        return Country
    else:
        return Country
    
def Mexican_sort(Country):
    Country = str(Country)
    if Country.lower() == 'méxico' or Country.lower() == 'mexico':
        Country = 'Mexico'
        return Country
    else:
        return Country
    
def German_sort(Country):
    if Country.lower() == 'germany' or Country.lower() == 'germany ':
        Country = 'Germany'
        return Country
    else:
        return Country 

def others_sort(Country):
    if Country == 'USA' or Country == 'UK' or Country == 'Germany' or Country == 'Canada' or Country == 'Australia and New Zealand' or Country == 'France' or Country == 'Brazil' or Country == 'Russia' or Country == 'Poland' or Country == 'The Netherlands' or Country == 'Latvia' or Country == 'Mexico':
        return Country
    else:
        Country = 'Other'
        return Country
    
data.Country = data.Country.apply(US_sort)
data.Country = data.Country.apply(UK_sort)
data.Country = data.Country.apply(Dutch_sort)
data.Country = data.Country.apply(Canadian_sort)
data.Country = data.Country.apply(French_sort)
data.Country = data.Country.apply(Russian_sort)
data.Country = data.Country.apply(Oceanic_sort)
data.Country = data.Country.apply(Mexican_sort)
data.Country = data.Country.apply(German_sort)
data.Country = data.Country.apply(others_sort)
country_count = data.Country.value_counts()
country_count

USA                          112
Other                         36
UK                            20
Germany                       15
Canada                        13
Australia and New Zealand      9
France                         8
Brazil                         8
Poland                         6
Russia                         6
The Netherlands                6
Latvia                         4
Mexico                         4
Name: Country, dtype: int64

In [7]:
data['View_on_sexuality'].loc[data.View_on_sexuality == 'Neutral'] = 4
data['View_on_sexuality'].loc[data.View_on_sexuality == 'Mostly positive'] = 6
data['View_on_sexuality'].loc[data.View_on_sexuality == 'Sometimes positive'] = 5
data['View_on_sexuality'].loc[data.View_on_sexuality == 'Absolutely positive'] = 7
data['View_on_sexuality'].loc[data.View_on_sexuality == 'Mostly negative'] = 2
data['View_on_sexuality'].loc[data.View_on_sexuality == 'Absolutely negative'] = 1
data['View_on_sexuality'].loc[data.View_on_sexuality == 'Sometimes negative'] = 3

C:\Users\d\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
data['Genre'][data.Genre.str.contains("Don't")] = 'None'
data['Genre'][data.Genre.str.contains("I like")] = 'Action/Adventure/Singleplayer RPGs/Multiplayer RPGs/Strategy'
data['Genre'][data.Genre.str.contains("Sim")] = 'Simulators'
data['Genre'][data.Genre.str.contains("Rhythm")] = 'Action'
data['Genre'][data.Genre.str.contains("Sand")] = 'Simulators'
data['Genre'][data.Genre.str.contains('F')] = 'Action'
data['Genre'][data.Genre.str.contains('strategy')] = 'Strategy'
data['Genre'][data.Genre.str.contains('all of')] = 'Action/Adventure/Singleplayer RPGs/Multiplayer RPGs/Simulators/Strategy'
data['Genre'][data.Genre.str.contains('vague')] = 'None'
data['Genre'][data.Genre.str.contains('Survival')] = 'Simulators'
data['Genre'][data.Genre.str.contains('every')] = 'Action/Adventure/Singleplayer RPGs/Multiplayer RPGs/Simulators/Strategy'
data['Genre'][data.Genre.str.contains('Prefer')] = 'None'
data['Genre'][data.Genre.str.contains('band')] = 'None'
data['Genre'][data.Genre.str.contains('Co')] = 'None'

<ipython-input-8-860eef040721>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Genre'][data.Genre.str.contains("Don't")] = 'None'
<ipython-input-8-860eef040721>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Genre'][data.Genre.str.contains("I like")] = 'Action/Adventure/Singleplayer RPGs/Multiplayer RPGs/Strategy'
<ipython-input-8-860eef040721>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Genre'][data.Genre.str

In [9]:
data['Genre'] = data['Genre'].apply(separation)
genres_exploded = data.explode('Genre')
genres_exploded.Genre.value_counts()

Singleplayer RPGs    90
Strategy             44
Action               41
Multiplayer RPGs     30
Adventure            28
Simulators           23
None                  6
Name: Genre, dtype: int64

In [10]:
data['Sexual_orientation'][data.Sexual_orientation.str.contains('Asex')] = 'Asexual'


<ipython-input-10-e809a269dc02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sexual_orientation'][data.Sexual_orientation.str.contains('Asex')] = 'Asexual'


In [11]:
def sexual_sort(Sexual_orientation):
    if Sexual_orientation in ['Heterosexual','Bisexual','Homosexual','Asexual']:
        return Sexual_orientation
    else:
        Sexual_orientation = 'Other'
        return Sexual_orientation
    

data.Sexual_orientation = data.Sexual_orientation.apply(sexual_sort)
data.Sexual_orientation.value_counts()

Heterosexual    154
Bisexual         49
Homosexual       17
Asexual          14
Other            13
Name: Sexual_orientation, dtype: int64

In [12]:
def gender_sort(Gender):
    if Gender in ['Male','Female']:
        return Gender
    else:
        Gender = 'Other'
        return Gender

data.Gender = data.Gender.apply(gender_sort)
data.Gender.value_counts()

Male      190
Female     40
Other      17
Name: Gender, dtype: int64

In [16]:
data['Other_interests'] = data['Other_interests'].apply(coma_separation)
interests_exploded = data.explode('Other_interests')
interests_exploded.Other_interests.value_counts()

["['Books(Non asian)'"                152
"'Anime'"                             103
"'Films and TV series'"                92
"'Manga'"                              76
"'Tabletop RPGs(D&D and similar)'"     73
                                     ... 
"'Philosophy'"                          1
"'Needlecraft-related hobbies']"]       1
"'Plants'"                              1
"'Technology']"]                        1
["['Electronics'"                       1
Name: Other_interests, Length: 102, dtype: int64

In [14]:
data

,date,Gender,Sexual_orientation,Age_group,Country,Regularity_of_gaming,Session_length,Genre,Favourite_game,Other_interests,View_on_sexuality,View_on_sexualisation,Design_preferences,Design_issues,View_on_sexual_sub_context,bruh,LGBT_representation,extras
0,04/01/2021 18:22:52,Other,Other,27-35,Latvia,Every day,5-8 hours,Multiplayer RPGs,"WoW, Lol, Tekken","[Books(Non asian), Tabletop games(just the bas...",5,4,NaN,Oversized breasts,3,Yes,"Yes, but only if explained logically and-or ex...",NaN
1,04/01/2021 18:43:27,Male,Heterosexual,21-26,Russia,Several times a week,0-4 hours,Simulators,"Deus Ex: Human Revolution, Prey (2017), Mass E...","[Books(Non asian), Films and TV series]",5,4,When it makes sense in a given setting,When there is no sense or the explanation is w...,4,Yes,"Yes, but only if explained logically and-or ex...",There are moments when erotic scenes are just ...
2,04/01/2021 18:43:43,Male,Heterosexual,21-26,Russia,Several times a week,0-4 hours,Singleplayer RPGs,Red Dead Redemption 2,"[Books(Non asian), Films and TV series, Manga]",6,7,NaN,NaN,5,Yes,"Yes, but only if explained logically and-or ex...",NaN
3,04/01/2021 18:52:43,Male,Heterosexual,27-35,Russia,Every day,5-8 hours,None,"Ace Combat 7, SpecOps The Line, Deus Ex Human ...","[Books(Non asian), Tabletop RPGs(D&D and simil...",4,4,marks of their mind characteristics,"Extreme sexualisation, too bright colors, hype...",4,Yes,Maybe/Neutral,NaN
4,04/01/2021 19:04:46,Male,Heterosexual,21-26,Latvia,Ranges from every day to several times a week.,5-8 hours,Action,Devil May Cry; Dark Souls; Tekken; Escape From...,"[Tabletop games(just the basic ones), Films an...",5,4,When the design of the character serves to tel...,Dislike when the creative process of designing...,4,Yes,If that type of content feels forced on then n...,As the LGBT+ community has gained more tractio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,05/01/2021 11:37:11,Male,Heterosexual,21-26,USA,Several times a week,0-4 hours,Multiplayer RPGs,"Halo, need for speed: underground, overwatch","[Books(Non asian), Films and TV series, Anime]",3,3,NaN,NaN,4,Yes,"Yes, in any way they appear there",NaN
243,05/01/2021 11:37:39,Male,Heterosexual,More than 35,Germany,Several times a week,0-4 hours,Action,"Celeste, Doom, Super Mario Galaxy, Firewatch, ...","[Books(Non asian), Tabletop games(just the bas...",4,2,I like pretty faces and generally attractive (...,"exaggerated boobs, unrealisticly skimpy outfit...",5,"Maybe, but if it's nonsense like Quiet's outfi...","Yes, in any way they appear there","I like playing games with attractive people, b..."
244,05/01/2021 11:43:50,Female,Heterosexual,27-35,USA,Several times a week,0-4 hours,Singleplayer RPGs,Skyrim,"[Books(Non asian), Films and TV series]",4,2,NaN,"Sexualized female characters, non sexualized m...",4,It is too overdone to be ok with context,"Yes, in any way they appear there",NaN
245,05/01/2021 11:45:41,Male,Heterosexual,Younger than 16,Other,Several times a week,0-4 hours,Simulators,"Democracy 4, Cities: Skylines","[Books(Non asian), Tabletop games(just the bas...",4,3,NaN,NaN,5,Yes,"Yes, in any way they appear there",NaN
